In [42]:
import sparse

In [58]:
import numpy

In [159]:
import scipy.sparse
import scipy.sparse.csgraph

In [160]:
from tmol.kinematics.builder import KinematicBuilder

In [161]:
from tmol.tests.conftest import ubq_system

In [267]:
sys = ubq_system()

In [381]:
import numba

@numba.jit(nopython=True, nogil=True)
def find_unsorted(coords, allow_dup):
    """Find first non-sorted entry in a [radix, ncoord] coord array."""
    radix, ncoords = coords.shape
    
    
    for c in range(ncoords-1):
        all_equal = True
        for r in range(radix):
            if coords[r, c] > coords[r, c+1]:
                return c
            elif coords[r, c] < coords[r, c+1]:
                all_equal = False
                break
                
        if all_equal and not allow_dup:
            return 
        
    return None

def is_sorted(array, allow_dup = False):
    """Check if a [radix, ncoord] sparse coord array is sorted."""
    if find_unsorted(array, allow_dup) is not None:
        return False
    else:
        return True

In [391]:
bg = sparse.COO(sys.bonds.T, data = numpy.full(len(sys.bonds), True), cache=True)

In [398]:
bg.shape

(1471, 1471)

In [396]:
bpl = scipy.sparse.csgraph.dijkstra(bg.tocsr(), directed=False, unweighted=True,  limit=3)

In [397]:
bpl[:15, :15]

array([[ 0.,  1.,  2.,  3.,  2.,  3., inf, inf,  1.,  2.,  3.,  3., inf,
        inf, inf],
       [ 1.,  0.,  1.,  2.,  1.,  2.,  3., inf,  2.,  1.,  2.,  2.,  3.,
         3., inf],
       [ 2.,  1.,  0.,  1.,  2.,  3., inf, inf,  3.,  2.,  3.,  3., inf,
        inf, inf],
       [ 3.,  2.,  1.,  0.,  3., inf, inf, inf, inf,  3., inf, inf, inf,
        inf, inf],
       [ 2.,  1.,  2.,  3.,  0.,  1.,  2.,  3.,  3.,  2.,  1.,  1.,  2.,
         2., inf],
       [ 3.,  2.,  3., inf,  1.,  0.,  1.,  2., inf,  3.,  2.,  2.,  1.,
         1.,  3.],
       [inf,  3., inf, inf,  2.,  1.,  0.,  1., inf, inf,  3.,  3.,  2.,
         2.,  2.],
       [inf, inf, inf, inf,  3.,  2.,  1.,  0., inf, inf, inf, inf,  3.,
         3.,  1.],
       [ 1.,  2.,  3., inf,  3., inf, inf, inf,  0.,  3., inf, inf, inf,
        inf, inf],
       [ 2.,  1.,  2.,  3.,  2.,  3., inf, inf,  3.,  0.,  3.,  3., inf,
        inf, inf],
       [ 3.,  2.,  3., inf,  1.,  2.,  3., inf, inf,  3.,  0.,  2.,  3.,
       